In [6]:
import time
import sys
import config
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import requests
from difflib import SequenceMatcher
from requests_html import HTMLSession
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
from tkinter import *
from tkinter import ttk
from tkinter import messagebox as mb
import tkinter as tk

In [9]:
class palace_bot:
    
    def __init__(self):
        self.base_url = base_url = 'https://shop.palaceskateboards.com'
        self.new_items_url = f'{base_url}/collections/new'
        self.new_items_url = 'https://shop.palaceskateboards.com/collections/t-shirts'
        self.desired_item = None
        self.desired_item_colour = None
        self.size = None
        self.basket = []
        self.item_urls = []
        self.season_range_items_name = []
        self.season_range_items = []
        self.size_list = []
        self.cop_list = []
        self.product_found = False
    
    def place_products_in_cop_list(self):
        temp_list = []
        temp_list.append(self.desired_item.get())
        temp_list.append(self.desired_item_colour.get())
        temp_list.append(self.size.get())
        self.cop_list.append(temp_list)
        cop_button.config(state='normal')
        self.desired_item_colour.set(None)
        self.size.set(None)

        
    def display_cop_list(self):
        cop_list_string = ""
        for item in self.cop_list:
            cop_list_string += f"{item[0].strip()}, {item[1].strip()}, {item[2].strip()}\n"
        mb.showinfo('Cop List', cop_list_string)
        
    def populate_size_list(self):
        for product in self.cop_list:
            self.size_list.append(product[2])
    
    def find_products_in_new(self):
        session = HTMLSession() 
        timeout = time.time() + (60 * 5)
        while len(self.basket) < 1:
            for num in range(1,6):
                response = session.get(f'{self.new_items_url}?page={num}')
                products = response.html.find('.product-grid-item.clearfix')
                try:
                    for product in products:
                        product_name = product.find('.title', first=True).text
                        for cop_item in self.cop_list:
                            if cop_item[0].strip() in product_name and cop_item[1].strip() in product_name:
                                self.basket.append(product) 
                                self.product_found = True
                except:
                    pass
        self.fetch_desired_item_urls_from_basket()
        
    def fetch_desired_item_urls_from_basket(self):
        for product in self.basket:
            product_url = product.find('.product-link', first=True)
            product_url_formatted = product_url.xpath('//a/@href', first=True)
            self.item_urls.append(f'{self.base_url}{product_url_formatted}')

    def find_all_clothes(self):
        range_url = 'https://www.palaceskateboards.com/range/ultimo-2020/'
        season_range_items_name = []
        season_range_items = []
        session = HTMLSession()
        response = session.get(range_url)
        products = response.html.find('.collection.clearfix')

        for product in products:
            product_name = product.find('.title', first=True).text
            self.season_range_items_name.append(product_name)
            self.season_range_items.append(product)
            
        self.season_range_items_name = sorted(self.season_range_items_name)
        
            
    def find_all_colours(self):
        global clothes_entry
        global colour_list
        session = HTMLSession()
        
        self.find_all_clothes()
        self.desired_item = clothes_entry_menu
        colour_list = []
        for product in self.season_range_items:
            if product.find('.title', first=True).text == self.desired_item.get():
                product_url = product.find('a', first=True).attrs['href']
                response = session.get(product_url)
                names = response.html.find('.slide')
                for name in names:
                    colour_and_product = str(name).split('data-name=\'')[1].split('\'')[0] #god forgive me
                    colour = colour_and_product.replace(self.desired_item.get(), "")
                    colour_list.append(colour)
        colour_list = set(colour_list)
        self.refresh_colours()
        return colour_list

    def refresh_colours(self):
        global colour_entry_menu
        global size_entry_menu
        global colour_entry
        
        menu = colour_entry_menu["menu"]
        menu.delete(0, END)
        for colour in colour_list:
                menu.add_command(label=colour, command=lambda value=colour:self.desired_item_colour.set(value))
        self.desired_item_colour.set(None)
        
    def refresh_sizes(self):
        global item_type
        global size_entry_menu
        
        menu = size_entry_menu["menu"]
        menu.delete(0, END)
        if item_type.get() == "Clothing":
            size_list = ['Small', 'Medium', 'Large', 'X-Large'] #Make sure this doesn't crash program
        elif item_type.get() == "Bags & Accessories":
            size_list = ['One Size']
        elif item_type.get() == "Boards":
            size_list = ['B1', 'B2', 'B3']
            
        for size in size_list:
                menu.add_command(label=size, command=lambda value=size:self.size.set(value))
        
    def execute_selenium_bot(self):
        
        timeout = time.time() + (60 * .2)
        while not self.product_found and time.time() < timeout:
            self.find_products_in_new()
            
        if not self.product_found:
            mb.showwarning('Timeout', "Product not found. Bot timed out.")
            sys.exit(0)
        else:
            self.populate_size_list()
            
            option = webdriver.ChromeOptions()
            option.add_argument('--disable-blink-features=AutomationControlled')
            option.add_argument('start-maximized')
            option.add_experimental_option("excludeSwitches", ["enable-automation"])
            option.add_experimental_option('useAutomationExtension', False) #TODO make agent more unique?
            agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36"
            option.add_argument(f"user-agent={agent}")
            driver = webdriver.Chrome(executable_path="E:/Downloads/chromedriver.exe", chrome_options=option)
            driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")          

            for x in range(len(self.item_urls)):
                driver.get(self.item_urls[x])
                WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, f"//select[@name='id']/option[text()='{self.size_list[x]}']"))).click()
                WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//input[@type='submit' and @value='Add to Cart']"))).click()
                time.sleep(.1)
                
            #Go to cart --> Takes a split second for items to appear
            time.sleep(.4)
            driver.get(f'{self.base_url}/cart')

            #Agree with T&Cs and proceed
            WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//input[@id='terms-checkbox']")))
            driver.execute_script("arguments[0].click();", driver.find_element_by_xpath("//input[@id='terms-checkbox']"))
            driver.find_element_by_id('checkout').click()

            #Delivery info
            driver.find_element_by_id('checkout_email').send_keys("test@test.com")
            driver.find_element_by_id('checkout_shipping_address_first_name').send_keys("John")
            driver.find_element_by_id('checkout_shipping_address_last_name').send_keys("Doe")
            driver.find_element_by_id('checkout_shipping_address_address1').send_keys("1 Palace Road")
            driver.find_element_by_id('checkout_shipping_address_city').send_keys("London")
            driver.find_element_by_id('checkout_shipping_address_zip').send_keys("BR1 1AA")
            driver.find_element_by_id('checkout_shipping_address_phone').send_keys("0123456789")

            #Click on captcha
            iframes = driver.find_elements_by_tag_name("iframe")
            driver.switch_to.frame(iframes[1])
            driver.find_element_by_id('recaptcha-anchor').click()

            time.sleep(5) #Captcha solving time (want to get rid off this)
            driver.switch_to.default_content()
            driver.find_element_by_id('continue_button').click()

            driver.find_element_by_id('continue_button').click()

            time.sleep(2) #Having to sleep because code below isn't working
            # WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//input[@type=submit and @id='continue_button']")))

            #Card information    
            iframes = driver.find_elements_by_tag_name("iframe")
            driver.switch_to.frame(iframes[1])
            driver.find_element_by_id('number').send_keys("1234 1234 1234")
            driver.switch_to.default_content()
            driver.switch_to.frame(iframes[2])
            driver.find_element_by_id('name').send_keys("MR JOHN DOE")
            driver.switch_to.default_content()
            driver.switch_to.frame(iframes[4])
            driver.find_element_by_id('expiry').send_keys("01")
            driver.find_element_by_id('expiry').send_keys("25")
            driver.switch_to.default_content()
            driver.switch_to.frame(iframes[5])
            driver.find_element_by_id('verification_value').send_keys("123")

            #Complete purchase
            driver.switch_to.default_content()
            driver.find_element_by_id('continue_button').click()

In [10]:
pb = palace_bot()

window = Tk()
window.configure(background='#FFEFDB')
item_type_list = ['Clothing', 'Bags & Accessories', 'Boards']
size_list = [None]
colour_list=[None]

global colour_list
global clothes_entry
global size_list
global colour_entry_menu
global size_entry_menu
global item_type_menu
global item_type



pb.find_all_clothes()


window.title("Palace Copper 1.0")
window.geometry("500x300")
text = Label(window, bg='#FFEFDB', font=('courier', 12, 'bold'), text="Palace Copper 1.0")
text.pack()
item_selection = Label(window, bg='#FFEFDB', text='Select an ITEM:')
item_selection.place(x=0, y=38)
pb.desired_item = StringVar(window)
pb.desired_item.set(None)
clothes_entry_menu = ttk.Combobox(window, values=pb.season_range_items_name, width = 35)

clothes_entry_menu.place(x=100, y=38)
item_selection_button = Button(window, text="Enter", command=pb.find_all_colours)
item_selection_button.place(x=340, y=35)

colour_selection = Label(window, bg='#FFEFDB', text='Select a COLOUR:')
colour_selection.place(x=0, y=80)
pb.desired_item_colour = StringVar(window)
pb.desired_item_colour.set(None)
colour_entry_menu = OptionMenu(window, pb.desired_item_colour, *colour_list)
colour_entry_menu.place(x=100, y=76)

item_type_selection = Label(window, bg='#FFEFDB', text='Select item type:')
item_type_selection.place(x=0, y=122)
item_type = StringVar(window)
item_type.set(None)
item_type_menu = OptionMenu(window, item_type, *item_type_list)
item_type_menu.place(x=100, y=118)

size_selection_button = Button(window, text="Enter", command=pb.refresh_sizes)
size_selection_button.place(x=250, y=120)

size_selection = Label(window, bg='#FFEFDB', text='Select a SIZE:')
size_selection.place(x=0, y=162)
pb.size = StringVar(window)
pb.size.set(None)
size_entry_menu = OptionMenu(window, pb.size, *size_list)
size_entry_menu.place(x=100, y=158)

add_item_to_cop_list_button = Button(window, text="Add to cop list", command=pb.place_products_in_cop_list)
add_item_to_cop_list_button.config(height =2, width=32)
add_item_to_cop_list_button.place(x=6, y=200)

print_cop_list_button = Button(window, text="View your cop list", command=pb.display_cop_list)
print_cop_list_button.config(height =2, width=32)
print_cop_list_button.place(x=256, y=200)

cop_button = Button(window, text="COP", state=DISABLED, command = pb.execute_selenium_bot)
cop_button.config(height =2, width=68)
cop_button.place(x=5, y=250)

window.mainloop()

E:\Python\Anaconda\lib\site-packages\ipykernel_launcher.py:146: DeprecationWarning: use options instead of chrome_options
